In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
import keras
from IPython.display import SVG
from keras.optimizers import Adam
from keras.utils.vis_utils import model_to_dot
from sklearn.model_selection import train_test_split
import lib
import pickle

Using TensorFlow backend.


In [38]:
def load_data(file_pref = '') -> (pd.DataFrame, pd.DataFrame):
    users = pd.read_csv(file_pref+'Users.csv', sep=';', dtype={'User_id': str, 'age': str})
    users.dropna(axis=0,inplace=True)
    users.age = pd.to_numeric(users.age, errors='coerce').fillna(30).astype(np.int64)
    users.loc[users.age>80, "age"] = 25
    users['User_id'] = users['User_id'].astype(np.int64)
    users.set_index('User_id', inplace=True)

    finder_decisions = pd.read_csv(file_pref+'Finder_decisions_.csv', sep=';')#, nrows=10000)
    finder_decisions.drop(finder_decisions[(~finder_decisions['Receiver_id'].isin(users.index))].index.values, inplace=True)
    finder_decisions.drop(finder_decisions[(~finder_decisions['Sender_id'].isin(users.index))].index.values, inplace=True)
    return users, finder_decisions

users, finder_decisions = load_data('')



(17592080, 4)
(16369297, 4)


In [179]:
print(finder_decisions.shape)
finder_decisions.head()

(16369297, 4)


,Decision,Number of Records,Receiver_id,Sender_id
0,like,1,3020120157,3019224295
1,like,1,3020143161,3021346119
2,like,1,3004593177,3021346119
3,like,1,3020177581,3021346119
4,like,1,3020193527,3019224295


In [56]:
print(users.shape)
users.head()

(82391, 4)


,Photo,age,gender,index
User_id,,,,
3025414901,https://r.phts.io/hilybu9d0lpf7mngw8gg8s8gwg8w...,30,1,0
3024640165,https://r.phts.io/hilybu44zkz4sc01ogsskc0480w8...,62,1,1
3022146285,https://r.phts.io/hilybu7gufdsc9wf8k444w84okkw...,23,2,2
3020392717,https://r.phts.io/hilybu760vygze57cwow0cswggkg...,20,2,3
3023036685,https://r.phts.io/hilybu5shyp459tl0ksgcws48sw0...,28,1,4


In [45]:
data=users
sender_value_counts = finder_decisions['Sender_id'].value_counts()
decisions =     finder_decisions.drop(
        finder_decisions.index[finder_decisions['Sender_id'].isin(sender_value_counts.index[sender_value_counts < 5])])
del sender_value_counts

In [60]:
users['key'] = np.arange(0,users.shape[0])
user_by_index=pd.DataFrame({"User_id":users.index.values}, users["key"])
index_by_user=pd.DataFrame({"key":user_by_index.index}, user_by_index.User_id)

users_data = users[['age', 'gender', 'key']]
decisions = decisions.merge(users_data, how='left', left_on='Sender_id', right_index=True)
decisions.rename(columns={'age': 'Sender_age', 'gender': 'Sender_gender', 'key': 'Sender_key'}, inplace=True)
decisions = decisions.merge(users_data, how='left', left_on='Receiver_id', right_index=True)
decisions.rename(columns={'age': 'Receiver_age', 'gender': 'Receiver_gender', 'key': 'Receiver_key'},inplace=True)
decisions['Decision'] = (decisions['Decision'] == 'like')+1

,User_id
key,
0,3025414901
1,3024640165
2,3022146285
3,3020392717
4,3023036685


In [119]:
decisions.head()users.head()

,Photo,age,gender,index,key
User_id,,,,,
3025414901,https://r.phts.io/hilybu9d0lpf7mngw8gg8s8gwg8w...,30,1,0,0
3024640165,https://r.phts.io/hilybu44zkz4sc01ogsskc0480w8...,62,1,1,1
3022146285,https://r.phts.io/hilybu7gufdsc9wf8k444w84okkw...,23,2,2,2
3020392717,https://r.phts.io/hilybu760vygze57cwow0cswggkg...,20,2,3,3
3023036685,https://r.phts.io/hilybu5shyp459tl0ksgcws48sw0...,28,1,4,4


In [126]:
decisions.head()

,Decision,Number of Records,Receiver_id,Sender_id,Receiver_key,Sender_key,Sender_age,Sender_gender,Receiver_age,Receiver_gender
0,2,1,3020120157,3019224295,71522,19310,33,1,33,2
1,2,1,3020143161,3021346119,22061,6526,19,2,23,1
2,2,1,3004593177,3021346119,75255,6526,19,2,22,1
3,2,1,3020177581,3021346119,15735,6526,19,2,20,1
4,2,1,3020193527,3019224295,66367,19310,33,1,20,2


In [ ]:
from sklearn.cluster import KMeans
from scipy.sparse import coo_matrix, lil_matrix

def extract_sparce_matrix(decisions):
    for (rindex, d) in decisions.iterrows():
         yield (d['Sender_key'], d['Receiver_key'], d['Decision'])

dim=users.shape[0]
S = lil_matrix((dim, dim))
for (i,j,d) in extract_sparce_matrix(decisions):
    S[i,j] = d

labeler = KMeans(n_clusters=15, max_iter=200, verbose=True)
labeler.fit(S.tocsr())

In [139]:
with open('labeler15clusters.pickle', 'wb') as f:
    pickle.dump(labeler, f)
    f.close()

In [140]:
clusters=pd.DataFrame({"cluster":labeler.labels_}, np.arange(0,len(labeler.labels_)))
decisions = decisions.merge(clusters, how='left', left_on='Sender_key', right_index=True)
users = users.merge(clusters, how='left', left_on='key', right_index=True)

In [184]:
cluster_reciever=decisions.groupby(["cluster", "Receiver_id"], as_index=False)["Decision"].mean()

answer = finder_decisions[["Decision","Receiver_id","Sender_id"]]
answer["Decision"] = (answer["Decision"] == "like") + 1
answer = answer.merge(users[["cluster"]], how="left", left_on="Sender_id", right_index=True )
answer.head()

,Decision,Receiver_id,Sender_id
0,2,3020120157,3019224295
1,2,3020143161,3021346119
2,2,3004593177,3021346119
3,2,3020177581,3021346119
4,2,3020193527,3019224295


In [207]:

answer2 = pd.merge(answer, cluster_reciever,  how='left', left_on=['cluster','Receiver_id'], right_on = ['cluster','Receiver_id'])

In [216]:
answer2[answer2.Decision_x == 1+(answer2.Decision_y>1.5)].shape[0]/answer2.shape[0]

0.7651222896132925